# **Human Action Recognition with MMSkeleton**
## Spatio-Tempoal Graph Convolutional Network (ST-GCN)


In [1]:
%%capture
!pip install wandb -qqq
import wandb

In [ ]:
%%writefile /root/.netrc
machine api.wandb.ai
  login user
  password 35238dfceafeb9e2e0695631c642d4b3913b0905

Writing /root/.netrc


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
Aborted!
^C


# Download datasets

In [3]:
%cd /content
file_id, file_name = "1_0fReh6XdE4Y0Q1-Wg1Mj1lrtTW5MPi-", "dataset_florence.zip"
!wget 'https://docs.google.com/uc?export=download&id=$file_id' -O $file_name
file_id, file_name = "1zUFdwiog47w88GKZAisJFg8GqD0nb8vD", "dataset_ccam.zip"
!wget 'https://docs.google.com/uc?export=download&id=$file_id' -O $file_name
#!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$file_id' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$file_id" -O $file_name && rm -rf ~/cookies.txt

/content
--2021-06-11 14:17:25--  https://docs.google.com/uc?export=download&id=1_0fReh6XdE4Y0Q1-Wg1Mj1lrtTW5MPi-
Resolving docs.google.com (docs.google.com)... 66.102.1.139, 66.102.1.113, 66.102.1.101, ...
Connecting to docs.google.com (docs.google.com)|66.102.1.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-2o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mt19v45qfdpb15n0fugoic8j5ui01dhf/1623421050000/13244527368295625576/*/1_0fReh6XdE4Y0Q1-Wg1Mj1lrtTW5MPi-?e=download [following]
--2021-06-11 14:17:32--  https://doc-0g-2o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mt19v45qfdpb15n0fugoic8j5ui01dhf/1623421050000/13244527368295625576/*/1_0fReh6XdE4Y0Q1-Wg1Mj1lrtTW5MPi-?e=download
Resolving doc-0g-2o-docs.googleusercontent.com (doc-0g-2o-docs.googleusercontent.com)... 74.125.206.132, 2a00:1450:400c:c04::84
Connecting to doc-0g-2o-docs.googleusercontent.com (doc-0g-2o-

# Functions

In [4]:
import cv2
from tqdm.notebook import trange, tqdm

def frame_iter(capture, description = ""):
  def _iterator():
    while capture.grab():
      yield capture.retrieve()

  return tqdm(
    _iterator(),
    desc=description,
    total=int(capture.get(cv2.CAP_PROP_FRAME_COUNT)),
    leave=False,
  )


def process_mmdet_results(mmdet_results, cat_id=0):
  """Process mmdet results, and return a list of bboxes.

  :param mmdet_results:
  :param cat_id: category id (default: 0 for human)
  :return: a list of detected bounding boxes
  """
  if isinstance(mmdet_results, tuple):
      det_results = mmdet_results[0]
  else:
      det_results = mmdet_results
  return det_results[cat_id]


# visualization
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

# Installation

## Install MMPose

In [ ]:
%%shell
pip install -q tqdm lazy-import
# This installation takes a long time due to the compiation of mmcv-full
pip install mmcv-full==1.0.5 -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
pip install mmdet==2.3.0

Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.7.0/index.html
     |████████████████████████████████| 235kB 12.4MB/s 
     |████████████████████████████████| 194kB 23.5MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.0.5-cp37-cp37m-linux_x86_64.whl size=17029367 sha256=180e78ce0a7de546beddd0c96c58b8f6e0f21694794ecd65ac2b13405e934be2
  Stored in directory: /root/.cache/pip/wheels/6b/63/58/2d7cf7e1eef88c034073cc7219fe39e9f2acc1bf4081db7a61
Successfully built mmcv-full
     |████████████████████████████████| 399kB 12.2MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=1de88bbacaddb083ee0c9512ceaae4286ccb1e4c017acf59845926525c800973
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [ ]:
%%shell
cd /content

git clone https://github.com/open-mmlab/mmpose.git -b v0.10.0
cd mmpose
pip install mmpose
# pip install -q -r requirements.txt
# python setup.py -q develop

Cloning into 'mmpose'...
remote: Enumerating objects: 9449, done.
remote: Counting objects: 100% (1316/1316), done.
remote: Compressing objects: 100% (539/539), done.
remote: Total 9449 (delta 887), reused 1039 (delta 763), pack-reused 8133
Receiving objects: 100% (9449/9449), 18.69 MiB | 1.93 MiB/s, done.
Resolving deltas: 100% (6163/6163), done.
Note: checking out '545738fce710b4db25bb03a7241a641c2afb0c94'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

     |████████████████████████████████| 348kB 13.8MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 317kB 22.8MB/s 
  Created wheel f

## Install MMSkeleton

In [ ]:
%%shell
cd /content

git clone https://github.com/open-mmlab/mmdetection.git -b v2.3.0 || true

Cloning into 'mmdetection'...
remote: Enumerating objects: 18375, done.
remote: Counting objects: 100% (464/464), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 18375 (delta 194), reused 223 (delta 107), pack-reused 17911
Receiving objects: 100% (18375/18375), 24.14 MiB | 19.22 MiB/s, done.
Resolving deltas: 100% (12675/12675), done.
Note: checking out '68d860d6f1a4a1fb687f2512038f2cbce7c6b08d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [ ]:
%%shell
cd /content
git clone https://github.com/open-mmlab/mmskeleton.git
cd mmskeleton
python setup.py develop

cd mmskeleton/ops/nms/
python setup_linux.py develop
cd ../../../



## File change and generation

### copy skeleton data in mmskeleton

In [ ]:
!mkdir -p /content/mmskeleton/data/
%cd /content/mmskeleton/data/
!unzip -qq /content/dataset_florence.zip
!unzip -qq /content/dataset_ccam.zip
%cd /content/mmskeleton/

/content/mmskeleton/data
/content/mmskeleton


### mmskl.py



In [ ]:
%%writefile mmskl.py
import argparse
import os
import sys
import logging

import torch
import mmskeleton
from mmcv import Config
from mmskeleton.utils import call_obj, set_attr, get_attr
""" Configuration Structure

argparse_cfg:
  <shortcut_name 1>:
    bind_to: <full variable path>
    help: <help message>
  <shortcut_name 2>:
    ...

processor_cfg:
  name: <full processor path>
  ...

"""

config_shortcut = dict(
    pose_demo_HD='./configs/pose_estimation/pose_demo_HD.yaml',
    pose_demo='./configs/pose_estimation/pose_demo.yaml')


def parse_cfg():

    parser = argparse.ArgumentParser(description='Run a processor.')
    parser.add_argument('config', help='configuration file path')

    if len(sys.argv) <= 1:
        args = parser.parse_args()
        return
    if sys.argv[1] == '-h' or sys.argv[1] == '--help':
        args = parser.parse_args()
        return

    # pop positional args
    config_args = []
    tmp = []
    for i, arg in enumerate(sys.argv):
        if i > 1:
            if arg[0] != '-':
                tmp.append(i)
            else:
                break

    for i in tmp[::-1]:
        config_args.append(sys.argv.pop(i))
    branch = config_args

    # load argument setting from configuration file
    if sys.argv[1] in config_shortcut:
        sys.argv[1] = config_shortcut[sys.argv[1]]

    print('Load configuration information from {}'.format(sys.argv[1]))
    cfg = Config.fromfile(sys.argv[1])
    for b in branch:
        if b in cfg:
            cfg = get_attr(cfg, b)
        else:
            print('The branch "{}" can not be found in {}.'.format(
                '-'.join(branch), sys.argv[1]))
            return dict()
    if 'description' in cfg:
        parser.description = cfg.description
    if 'argparse_cfg' not in cfg:
        cfg.argparse_cfg = dict()
    for key, info in cfg.argparse_cfg.items():
        if 'bind_to' not in info:
            continue
        default = get_attr(cfg, info['bind_to'])
        if 'type' not in info:
            if default is not None:
                info['type'] = type(default)
        else:
            info['type'] = eval(info['type'])
        kwargs = dict(default=default)
        kwargs.update({k: v for k, v in info.items() if k != 'bind_to'})
        parser.add_argument('--' + key, **kwargs)
    args = parser.parse_args()

    # update config from command line
    for key, info in cfg.argparse_cfg.items():
        if 'bind_to' not in info:
            continue
        value = getattr(args, key)
        set_attr(cfg, info['bind_to'], value)

    # replace pre_defined arguments in configuration files
    def replace(cfg, **format_args):
        if isinstance(cfg, str):
            return cfg.format(**format_args)
        if isinstance(cfg, dict):
            for k, v in cfg.items():
                set_attr(cfg, k, replace(v, **format_args))
        elif isinstance(cfg, list):
            for k in range(len(cfg)):
                cfg[k] = replace(cfg[k], **format_args)
        return cfg

    format_args = dict()
    format_args['config_path'] = args.config
    format_args['config_name'] = os.path.basename(format_args['config_path'])
    format_args['config_prefix'] = format_args['config_name'].split('.')[0]
    cfg = replace(cfg, **format_args)
    return cfg


def main():
    cfg = parse_cfg()
    if 'processor_cfg' in cfg:
        call_obj(**cfg.processor_cfg)
    else:
        print('No processor specified.')


if __name__ == "__main__":
    torch.multiprocessing.set_start_method('spawn')
    main()


Overwriting mmskl.py


### mmskeleton/processor/recognition.py

In [ ]:
%%writefile mmskeleton/processor/recognition.py
from collections import OrderedDict
import torch
import logging
import numpy as np
from mmskeleton.utils import call_obj, import_obj, load_checkpoint
from mmcv.runner import EpochBasedRunner
from mmcv import Config, ProgressBar
from mmcv.parallel import MMDataParallel
from mmdet.utils import get_root_logger

import wandb

def test(model_cfg,
         dataset_cfg,
         checkpoint,
         batch_size=None,
         gpu_batch_size=None,
         gpus=-1,
         workers=4):

    # calculate batch size
    if gpus < 0:
        gpus = torch.cuda.device_count()
    if (batch_size is None) and (gpu_batch_size is not None):
        batch_size = gpu_batch_size * gpus
    assert batch_size is not None, 'Please appoint batch_size or gpu_batch_size.'

    dataset = call_obj(**dataset_cfg)
    data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                              batch_size=batch_size,
                                              shuffle=False,
                                              num_workers=workers)

    # put model on gpus
    if isinstance(model_cfg, list):
        model = [call_obj(**c) for c in model_cfg]
        model = torch.nn.Sequential(*model)
    else:
        model = call_obj(**model_cfg)
    load_checkpoint(model, checkpoint, map_location='cpu')
    model = MMDataParallel(model, device_ids=range(gpus)).cuda()
    model.eval()

    results = []
    labels = []
    prog_bar = ProgressBar(len(dataset))
    for data, label in data_loader:
        with torch.no_grad():
            output = model(data)
            output = model(data).data.cpu().numpy()

        results.append(output)
        labels.append(label)
        for i in range(len(data)):
            prog_bar.update()
    results = np.concatenate(results)
    labels = np.concatenate(labels)
    
    print()
    print(np.argmax(results, axis=1))
    print(labels)

    print('Top 1: {:.2f}%'.format(100 * topk_accuracy(results, labels, 1)))
    print('Top 3: {:.2f}%'.format(100 * topk_accuracy(results, labels, 3)))


def train(
        work_dir,
        model_cfg,
        loss_cfg,
        dataset_cfg,
        optimizer_cfg,
        total_epochs,
        training_hooks,
        batch_size=None,
        gpu_batch_size=None,
        workflow=[('train', 1)],
        gpus=-1,
        log_level=0,
        workers=4,
        resume_from=None,
        load_from=None,
):
    global loss
    logger = get_root_logger(log_level=log_level)
    config = {
      'workdir': work_dir,
      'model_cfg': model_cfg,
      'loss_cfg': loss_cfg,
      'dataset_cfg': dataset_cfg,
      'optimizer_cfg': optimizer_cfg,
      'total_epochs': total_epochs,
      'training_hooks': training_hooks,
    }

    with wandb.init(project="STGCN", config=config):
      # calculate batch size
      if gpus < 0:
          gpus = torch.cuda.device_count()
      if (batch_size is None) and (gpu_batch_size is not None):
          batch_size = gpu_batch_size * gpus
      assert batch_size is not None, 'Please appoint batch_size or gpu_batch_size.'

      # prepare data loaders
      if isinstance(dataset_cfg, dict):
          dataset_cfg = [dataset_cfg]
      data_loaders = [
          torch.utils.data.DataLoader(dataset=call_obj(**d),
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=workers,
                                      drop_last=True) for d in dataset_cfg
      ]

    
      # put model on gpus
      if isinstance(model_cfg, list):
          model = [call_obj(**c) for c in model_cfg]
          model = torch.nn.Sequential(*model)
      else:
          model = call_obj(**model_cfg)
      model.apply(weights_init)

      # Magic
      wandb.watch(model)

      model = MMDataParallel(model, device_ids=range(gpus)).cuda()
      loss = call_obj(**loss_cfg)


      # build runner
      optimizer = call_obj(params=model.parameters(), **optimizer_cfg)
      runner = EpochBasedRunner(model, batch_processor, optimizer, work_dir, logger)
      runner.register_training_hooks(**training_hooks)

      if resume_from:
          runner.resume(resume_from)
      elif load_from:
          runner.load_checkpoint(load_from)

      # run
      workflow = [tuple(w) for w in workflow]
      runner.run(data_loaders, workflow, total_epochs, loss=loss)


# process a batch of data
def batch_processor(model, datas, train_mode):
    global loss

    data, label = datas
    data = data.cuda()
    label = label.cuda()

    # forward
    output = model(data)
    losses = loss(output, label)

    # output
    log_vars = dict(loss=losses.item())
    if not train_mode:
        log_vars['top1'] = topk_accuracy(output, label)
        log_vars['top3'] = topk_accuracy(output, label, 3)

    outputs = dict(loss=losses, log_vars=log_vars, num_samples=len(data.data))
    wandb.log(log_vars)
    return outputs


def topk_accuracy(score, label, k=1):
    rank = score.argsort()
    hit_top_k = [l in rank[i, -k:] for i, l in enumerate(label)]
    accuracy = sum(hit_top_k) * 1.0 / len(hit_top_k)
    return accuracy


def weights_init(model):
    classname = model.__class__.__name__
    if classname.find('Conv1d') != -1:
        model.weight.data.normal_(0.0, 0.02)
        if model.bias is not None:
            model.bias.data.fill_(0)
    elif classname.find('Conv2d') != -1:
        model.weight.data.normal_(0.0, 0.02)
        if model.bias is not None:
            model.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)


### configs/pose_estimation/pose_demo.yaml

In [ ]:
%%writefile configs/pose_estimation/pose_demo.yaml
processor_cfg:
  type: "processor.pose_demo.inference"
  gpus: 1
  worker_per_gpu: 1
  video_file: resource/data_example/skateboarding.mp4
  save_dir: "work_dir/pose_demo"

  detection_cfg:
    model_cfg: ../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  video:
    bind_to: processor_cfg.video_file
    help: path to input video
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file


### configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml

In [ ]:
%%writefile configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml
detection_cfg:
  model_cfg: ../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
  checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
  bbox_thre: 0.8
estimation_cfg:
  model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
  checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
  data_cfg:
    image_size:
      - 192
      - 256
    pixel_std: 200
    image_mean:
      - 0.485
      - 0.456
      - 0.406
    image_std:
      - 0.229
      - 0.224
      - 0.225
    post_process: true


Overwriting configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml


### configs/utils/build_dataset_example.yaml

In [ ]:
%%writefile configs/utils/build_dataset_example.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 1
  video_dir: resource/data_example
  out_dir: "data/dataset_example"
  category_annotation: resource/category_annotation_example.json
  detection_cfg:
    model_cfg: ../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Overwriting configs/utils/build_dataset_example.yaml


### configs/utils/build_dataset_florence.yaml

In [ ]:
%%writefile configs/utils/build_dataset_florence.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 1
  video_dir: "../Florence_3d_actions"
  out_dir: "data/dataset_florence_ccam"
  category_annotation: resource/category_annotation_florence_ccam.json
  detection_cfg:
    model_cfg: ../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Writing configs/utils/build_dataset_florence.yaml


### configs/utils/build_dataset_ccam.yaml

In [ ]:
%%writefile configs/utils/build_dataset_ccam.yaml
processor_cfg:
  type: "processor.skeleton_dataset.build"
  gpus: 1
  worker_per_gpu: 1
  video_dir: "../ccam_actions"
  out_dir: "data/dataset_florence_ccam"
  category_annotation: resource/category_annotation_florence_ccam.json
  detection_cfg:
    model_cfg: ../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py
    checkpoint_file: http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth
    bbox_thre: 0.8
  estimation_cfg:
    model_cfg: configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml
    checkpoint_file: mmskeleton://pose_estimation/pose_hrnet_w32_256x192
    data_cfg:
      image_size:
        - 192
        - 256
      pixel_std: 200
      image_mean:
        - 0.485
        - 0.456
        - 0.406
      image_std:
        - 0.229
        - 0.224
        - 0.225
      post_process: true
  tracker_cfg: null

argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  worker_per_gpu:
    bind_to: processor_cfg.worker_per_gpu
    help: number of workers for each gpu
  video_dir:
    bind_to: processor_cfg.video_dir
    help: folder for videos
  category_annotation:
    bind_to: processor_cfg.category_annotation
    help: a json file recording video category annotation
  out_dir:
    bind_to: processor_cfg.out_dir
    help: folder for storing output dataset
  skeleton_model:
    bind_to: processor_cfg.estimation_cfg.model_cfg
  skeleton_checkpoint:
    bind_to: processor_cfg.estimation_cfg.checkpoint_file
  detection_model:
    bind_to: processor_cfg.detection_cfg.model_cfg
  detection_checkpoint:
    bind_to: processor_cfg.detection_cfg.checkpoint_file

Writing configs/utils/build_dataset_ccam.yaml


### resource/category_annotation_florence_ccam.json

In [ ]:
%%writefile /content/mmskeleton/resource/category_annotation_florence_ccam.json
{
    "categories": [
        "wave",
        "drink from a bottle",
        "answer phone",
        "clap",
        "tight lace",
        "sit down",
        "stand up",
        "read watch",
        "bow",
        "coming in",
        "going out",
        "g230"
    ],
    "annotations": {
        "GestureRecording_Id15actor7idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor6idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id8actor9idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor5idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor9idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id7actor5idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id4actor5idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id5actor6idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor9idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id15actor1idAction15category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor5idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id4actor8idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor1idAction17category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor4idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id19actor4idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id22actor6idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor7idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id25actor1idAction25category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id1actor1idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor4idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id1actor2idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id13actor2idAction13category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id15actor5idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor2idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id11actor7idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id28actor1idAction28category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor9idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor6idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id21actor6idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor5idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id10actor9idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id14actor4idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id5actor2idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id9actor2idAction9category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor10idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id6actor10idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id12actor10idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor1idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id14actor3idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id17actor10idAction17category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id26actor1idAction26category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id8actor7idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id11actor5idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor2idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id16actor2idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id4actor7idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id6actor3idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id2actor8idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor6idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id14actor2idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id12actor5idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id14actor7idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id1actor4idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id5actor1idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id4actor3idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id12actor1idAction12category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor9idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id16actor1idAction16category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id2actor3idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor6idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id8actor1idAction8category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor8idAction14category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id20actor5idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id2actor6idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id6actor2idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id1actor3idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor3idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor1idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor8idAction18category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id3actor3idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id23actor8idAction23category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id4actor9idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id16actor4idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id15actor10idAction15category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id7actor1idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id6actor4idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id18actor7idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor1idAction9category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id7actor8idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id14actor1idAction14category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id23actor1idAction23category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id12actor3idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id13actor9idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id18actor4idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id22actor1idAction22category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id9actor10idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id3actor5idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id17actor5idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor6idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id7actor7idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id13actor7idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id13actor10idAction13category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id17actor3idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor1idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id4actor10idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id21actor10idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor10idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id6actor7idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id15actor3idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id1actor8idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id22actor10idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id10actor5idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id16actor10idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor9idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id3actor10idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id20actor3idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id19actor3idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id4actor6idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id27actor1idAction27category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor10idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id6actor8idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id13actor8idAction13category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id8actor3idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor1idAction19category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id1actor9idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id18actor10idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor8idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id11actor4idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id13actor3idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id2actor2idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id12actor8idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id9actor3idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id1actor6idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor8idAction16category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id8actor5idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor7idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id16actor5idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id17actor4idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id16actor3idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id3actor6idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id19actor10idAction19category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id9actor5idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id15actor6idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id3actor8idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id18actor1idAction18category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id6actor6idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id15actor4idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id7actor3idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id13actor6idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id19actor6idAction19category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id6actor5idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor6idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id10actor8idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id19actor8idAction19category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor7idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id13actor4idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id4actor1idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id8actor10idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id17actor2idAction17category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id20actor1idAction20category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id4actor4idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id22actor8idAction22category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id12actor2idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id11actor10idAction11category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id7actor10idAction7category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id5actor4idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id4actor2idAction4category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id12actor6idAction12category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id24actor1idAction24category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id14actor10idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id23actor10idAction23category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id18actor5idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id17actor7idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id6actor1idAction6category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id9actor7idAction9category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor4idAction3category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id16actor9idAction16category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id17actor6idAction17category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id5actor8idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id2actor9idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id12actor4idAction12category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id11actor1idAction11category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id7actor4idAction7category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id14actor9idAction14category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id8actor2idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id21actor8idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id11actor3idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id3actor9idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id8actor8idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id18actor9idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id8actor4idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id5actor9idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id29actor1idAction29category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id9actor9idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id13actor5idAction13category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id10actor2idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id16actor7idAction16category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id10actor10idAction10category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id15actor9idAction15category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id13actor1idAction13category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id1actor10idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id15actor2idAction15category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor2idAction11category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id15actor8idAction15category6.avi": {
          "category_id": 5
        },
        "GestureRecording_Id6actor9idAction6category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id9actor4idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id2actor7idAction2category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id20actor6idAction20category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id3actor7idAction3category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id18actor3idAction18category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id1actor7idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id20actor8idAction20category8.avi": {
          "category_id": 7
        },
        "GestureRecording_Id11actor9idAction11category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id9actor8idAction9category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id8actor6idAction8category4.avi": {
          "category_id": 3
        },
        "GestureRecording_Id18actor2idAction18category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id5actor3idAction5category2.avi": {
          "category_id": 1
        },
        "GestureRecording_Id1actor5idAction1category1.avi": {
          "category_id": 0
        },
        "GestureRecording_Id10actor6idAction10category5.avi": {
          "category_id": 4
        },
        "GestureRecording_Id14actor5idAction14category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id5actor7idAction5category3.avi": {
          "category_id": 2
        },
        "GestureRecording_Id21actor3idAction21category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id17actor8idAction17category7.avi": {
          "category_id": 6
        },
        "GestureRecording_Id24actor8idAction24category9.avi": {
          "category_id": 8
        },
        "GestureRecording_Id21actor1idAction21category7.avi": {
          "category_id": 6
        },
        "TightLace_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "Clapping_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "ReadWatch_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Stationary_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "TightLace_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "Clapping_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "ReadWatch_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "WaveHand_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Stationary_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "TightLace_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "ReadWatch_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "ReadWatch_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "Clapping_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "Bow_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "ReadWatch_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Bow_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "Bow_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "Stationary_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "Stationary_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Stationary_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "Stationary_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Bow_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "G230_Master_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "G230_Master_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "TightLace_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "Stationary_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "Bow_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "G230_Master_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "ReadWatch_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "WaveHand_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Clapping_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "Stationary_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "Bow_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "TightLace_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 6
        },
        "TightLace_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "AnswerPhone_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "Stationary_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "WaveHand_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Bow_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 7
        },
        "ReadWatch_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "TightLace_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 1
        },
        "Clapping_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "G230_Master_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "Bow_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "DrinkFromBottle_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Stationary_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "WaveHand_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "G230_Master_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 3
        },
        "TightLace_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 11
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 7
        },
        "Stationary_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "G230_Master_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "Bow_cam_rosberry3_color_image_raw_act1.mp4": {
          "category_id": 8
        },
        "G230_Master_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 11
        },
        "Stationary_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 2
        },
        "WaveHand_cam_rosberry3_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "ReadWatch_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry2_color_image_raw_act1.mp4": {
          "category_id": 4
        },
        "WaveHand_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "ReadWatch_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 2
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 2
        },
        "Bow_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "SitDownStandUp_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 5
        },
        "Clapping_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "AnswerPhone_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "ReadWatch_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry2_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry2_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "Clapping_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "WaveHand_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "Bow_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 8
        },
        "Clapping_cam_rosberry4_color_image_raw_act2.mp4": {
          "category_id": 3
        },
        "SitDownStandUp_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "TightLace_cam_rosberry2_color_image_raw_act3.mp4": {
          "category_id": 4
        },
        "G230_Master_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "AnswerPhone_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "Clapping_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry3_color_image_raw_act2.mp4": {
          "category_id": 0
        },
        "WaveHand_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "G230_Master_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "WaveHand_cam_rosberry4_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "TightLace_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 0
        },
        "WaveHand_cam_rosberry1_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "SitDownStandUp_cam_rosberry2_color_image_raw_act2.mp4": {
          "category_id": 5
        },
        "DrinkFromBottle_cam_rosberry3_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "G230_Master_cam_rosberry4_color_image_raw_act3.mp4": {
          "category_id": 11
        },
        "DrinkFromBottle_cam_rosberry1_color_image_raw_act3.mp4": {
          "category_id": 1
        },
        "DrinkFromBottle_cam_rosberry4_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "WaveHand_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 0
        },
        "Stationary_cam_rosberry1_color_image_raw_act1.mp4": {
          "category_id": 6
        },
        "Stationary_cam_rosberry3_color_image_raw_goingout.mp4": {
          "category_id": 10
        },
        "TightLace_cam_rosberry4_color_image_raw_comingin.mp4": {
          "category_id": 9
        },
        "SitDownStandUp_cam_rosberry1_color_image_raw_comingin.mp4": {
          "category_id": 9
        }
    }
}


Writing /content/mmskeleton/resource/category_annotation_florence_ccam.json


### configs/recognition/st_gcn/dataset_fc/train.yaml and test.yml

In [ ]:
!mkdir -p configs/recognition/st_gcn/dataset_fc/

In [ ]:
%%writefile configs/recognition/st_gcn/dataset_fc/train.yaml
argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  work_dir:
    bind_to: processor_cfg.work_dir
    help: the dir to save logs and models
  batch_size:
    bind_to: processor_cfg.batch_size
  resume_from:
    bind_to: processor_cfg.resume_from
    help: the checkpoint file to resume from


processor_cfg:
  type: 'processor.recognition.train'
  workers: 16

  # model setting
  model_cfg:
    type: 'models.backbones.ST_GCN_18'
    in_channels: 3
    num_class: 12
    edge_importance_weighting: True
    graph_cfg:
      layout: 'coco'
      strategy: 'spatial'
  loss_cfg:
    type: 'torch.nn.CrossEntropyLoss'

  # dataset setting
  dataset_cfg:
    # training set
    - type: "datasets.DataPipeline"
      data_source:
        type: "datasets.SkeletonLoader"
        data_dir: ./data/dataset_ccam
        num_track: 2
        num_keypoints: 17
        repeat: 20
      pipeline:
        - {type: "datasets.skeleton.normalize_by_resolution"}
        - {type: "datasets.skeleton.mask_by_visibility"}
        - {type: "datasets.skeleton.pad_zero", size: 150 }
        - {type: "datasets.skeleton.random_crop", size: 150 }
        - {type: "datasets.skeleton.simulate_camera_moving"}
        - {type: "datasets.skeleton.transpose", order: [0, 2, 1, 3]}
        - {type: "datasets.skeleton.to_tuple"}

    - type: "datasets.DataPipeline"
      data_source:
        type: "datasets.SkeletonLoader"
        data_dir: ./data/dataset_ccam
        num_track: 2
        num_keypoints: 17
      pipeline:
        - {type: "datasets.skeleton.normalize_by_resolution"}
        - {type: "datasets.skeleton.mask_by_visibility"}
        - {type: "datasets.skeleton.pad_zero", size: 300 }
        - {type: "datasets.skeleton.random_crop", size: 300 }
        - {type: "datasets.skeleton.transpose", order: [0, 2, 1, 3]}
        - {type: "datasets.skeleton.to_tuple"}

  # dataloader setting
  batch_size: 64
  gpus: 1

  # optimizer setting
  optimizer_cfg:
    type: 'torch.optim.SGD'
    lr: 0.01
    momentum: 0.9
    nesterov: true
    weight_decay: 0.0001

  # runtime setting
  workflow: [['train', 5], ['val', 1]]
  work_dir: ./work_dir/recognition/st_gcn/dataset_florence_ccam
  log_level: 0
  total_epochs: 80
  training_hooks:
    lr_config:
      policy: 'step'
      step: [20, 40, 50, 60]
    log_config:
      interval: 100
      hooks:
        - type: TextLoggerHook
    checkpoint_config:
      interval: 5
    optimizer_config:
      grad_clip:
  resume_from:
  load_from:

Writing configs/recognition/st_gcn/dataset_fc/train.yaml


In [ ]:
%%writefile configs/recognition/st_gcn/dataset_fc/test.yaml
argparse_cfg:
  gpus:
    bind_to: processor_cfg.gpus
    help: number of gpus
  batch_size:
    bind_to: processor_cfg.batch_size
  checkpoint:
    bind_to: processor_cfg.checkpoint
    help: the checkpoint file to load from

processor_cfg:
  type: "processor.recognition.test"
  workers: 16
  #checkpoint: ./work_dir/recognition/st_gcn/dataset_florence_ccam/epoch_10.pth
  checkpoint: ./work_dir/recognition/st_gcn/dataset_florence_ccam/latest.pth

  # model setting
  model_cfg:
    type: "models.backbones.ST_GCN_18"
    in_channels: 3
    num_class: 12
    edge_importance_weighting: True
    graph_cfg:
      layout: "coco"
      strategy: "spatial"

  # dataset setting
  dataset_cfg:
    type: "datasets.DataPipeline"
    data_source:
      type: "datasets.SkeletonLoader"
      data_dir: ./data/dataset_ccam
      num_track: 2
      num_keypoints: 17
    pipeline:
      - { type: "datasets.skeleton.normalize_by_resolution" }
      - { type: "datasets.skeleton.mask_by_visibility" }
      - { type: "datasets.skeleton.pad_zero", size: 300 }
      - { type: "datasets.skeleton.random_crop", size: 300 }
      - { type: "datasets.skeleton.transpose", order: [0, 2, 1, 3] }
      - { type: "datasets.skeleton.to_tuple" }
  # dataloader setting
  batch_size: null
  gpu_batch_size: 64
  gpus: -1


Writing configs/recognition/st_gcn/dataset_fc/test.yaml


## pose demo

In [ ]:
%cd /content/mmskeleton
# !mmskl pose_demo

/content/mmskeleton


In [ ]:
# import mmcv
# import mmdet
# from mmskeleton.apis import init_pose_estimator, inference_pose_estimator

# cfg = mmcv.Config.fromfile('configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml')
# # video = mmcv.VideoReader('resource/data_example/skateboarding.mp4')

# model = init_pose_estimator(**cfg, device=0)
# # for i, frame in enumerate(video):
# #   result = inference_pose_estimator(model, frame)
# #   print('Process the frame {}'.format(i))

Config (path: configs/apis/pose_estimator.cascade_rcnn+hrnet.yaml): {'detection_cfg': {'model_cfg': '../mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco.py', 'checkpoint_file': 'http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth', 'bbox_thre': 0.8}, 'estimation_cfg': {'model_cfg': 'configs/pose_estimation/hrnet/pose_hrnet_w32_256x192_test.yaml', 'checkpoint_file': 'mmskeleton://pose_estimation/pose_hrnet_w32_256x192', 'data_cfg': {'image_size': [192, 256], 'pixel_std': 200, 'image_mean': [0.485, 0.456, 0.406], 'image_std': [0.229, 0.224, 0.225], 'post_process': True}}}


Downloading: "http://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth" to /root/.cache/torch/hub/checkpoints/cascade_rcnn_r50_fpn_1x_coco_20200316-3dc56deb.pth


Downloading: "https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmskeleton/models/pose_estimation/pose_hrnet_w32_256x192-76ea353b.pth" to /root/.cache/torch/hub/checkpoints/pose_hrnet_w32_256x192-76ea353b.pth



Process the frame 0
Process the frame 1
Process the frame 2
Process the frame 3
Process the frame 4
Process the frame 5
Process the frame 6
Process the frame 7
Process the frame 8
Process the frame 9
Process the frame 10
Process the frame 11
Process the frame 12
Process the frame 13
Process the frame 14
Process the frame 15
Process the frame 16
Process the frame 17
Process the frame 18
Process the frame 19
Process the frame 20
Process the frame 21
Process the frame 22
Process the frame 23
Process the frame 24
Process the frame 25
Process the frame 26
Process the frame 27
Process the frame 28
Process the frame 29
Process the frame 30
Process the frame 31
Process the frame 32
Process the frame 33
Process the frame 34
Process the frame 35
Process the frame 36
Process the frame 37
Process the frame 38
Process the frame 39
Process the frame 40
Process the frame 41
Process the frame 42
Process the frame 43
Process the frame 44
Process the frame 45
Process the frame 46
Process the frame 47
P

## Run the pose estimation on the CCAM dataset.

In [ ]:
import glob

video_list = glob.glob('/content/mmskeleton/data/dataset_ccam/*.mp4')
video_list.sort(reverse=False)
print(video_list)
pose_estimation_results = {}
with tqdm(total=len(video_list)) as pbar:
  for video_path in video_list:
    filename = osp.basename(video_path)
    video = mmcv.VideoReader(filename)
    results = []
    for i, frame in enumerate(video):
      result = inference_pose_estimator(model, frame)
      results.append(result)
    pose_estimation_results[filename] = results
    pbar.update(1)
    if save_out_video:
      video_result = video_path.replace('mmskeleton/data/dataset_ccam/','video_results/vis_')
      video_result1 = video_result.replace('avi','mp4')
      !ffmpeg -y -loglevel panic -i $video_result -vcodec libx264 $video_result1
      !rm $video_result

[]


In [ ]:
import random
video_result_list = glob.glob('/content/video_results/*.mp4')
video_result = random.choice(video_result_list)
show_local_mp4_video(video_result)

In [ ]:
print(pose_estimation_results.keys())
pickle.dump(pose_estimation_results, open("/content/pose_estimation_results_poseresnet152.pkl", "wb"))

In [ ]:
# Change current working directory to /content
%cd /content
!zip -q -r video_results_poseresnet152.zip video_results

# Action Recognition


## model training

In [ ]:
%cd /content/mmskeleton
!mmskl configs/recognition/st_gcn/dataset_fc/train.yaml

## model test

In [ ]:
%cd /content/mmskeleton
!mmskl configs/recognition/st_gcn/dataset_fc/test.yaml

# Extra codes

In [ ]:
!rm -rf ./data/dataset_florence/.ipynb_checkpoints

In [ ]:
!zip -r dataset_ccam.zip /content/mmskeleton/data/dataset_ccam
!zip -r dataset_florence.zip /content/mmskeleton/data/dataset_florence